**Текущие задачи**:
- Реализация опроса новых данных о свежих статьях
- Описать `requirements` или сделать Docker-образ
- Выгружать датасет из памяти после того, как он был считан. Составлять новый датасет, выводить в csv и конкатенировать сами csv.

# Парсер

Реализованы парсеры данных, собраны тексты и метаинформация статей следующих ресурсов:
- Хабрахабр https://habr.com/
- Типичный программист https://tproger.ru/

Также реализован механизм докачивания новых статей, чтобы учесть появление новых технологий. Чтобы не создавать повышенную нагрузку на серверы ресурсов и не вызывать их подозрений, опрашивание ресурсов происходит поочередно. Методы парсеров содержатся внутри класса `Parser`, методы оркестровки (планирование и общий пайплайн вызова парсеров) – в классе `ParserComposer`.

Системные требования:
- Каталог с датасетами
- Дравйвер `/usr/bin/chromedriver`

In [1]:
# standard library
import random
import datetime
import pickle
import os
import re
import uuid

# conections
import requests
from requests.exceptions import ConnectionError

# tor network for private parsing with privoxy and stem
# https://gist.github.com/KhepryQuixote/46cf4f3b999d7f658853
from stem import Signal
from stem.control import Controller

# for tor network IP changing
with Controller.from_port(port = 9051) as controller:
    controller.authenticate('07011951')
    controller.signal(Signal.NEWNYM)

headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1) AppleWebKit/537.73.11 (KHTML, like Gecko) Version/7.0.1 Safari/537.73.11'
}

# parsers libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# data science 
import pandas as pd
from tqdm.notebook import tqdm, trange

sources = {
    'tproger':
    {
        'base_url': 'https://tproger.ru/',
        'first_page_url': 'https://tproger.ru/page/1/',
        'find_all_args': ('a', {'class':'article-link'})
    },
    'habr':
    {
        'base_url': 'https://habr.com/',
        'first_page_url': 'https://habr.com/ru/page1/',
        'find_all_args': ('a', {'class':'post__title_link'})
    },
#     'proglib':
#     {
#         'base_url': 'https://proglib.io/',
#         'first_page_url': 'https://proglib.io/',
#         'btn_class': 'load-more active',
#     }
}

proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050'
}


DATASETS_PATH = '/home/leo/DATASETS'

In [2]:
def k_to_num(s):
    if type(s) is str:
        s = s.replace(',', '.')
        if 'k' in s:
            s = s.replace('k', 'e3')
    return round(float(s))

class Parser:
    def __init__(self, source_name):
        self.source_name = source_name
        self.source = sources[source_name]
        self.df = pd.read_csv(f"{DATASETS_PATH}/{source_name}.csv",
                         index_col=0,
                         parse_dates=['post_time', 'parse_time'])

    
    def check_new_articles(self, nurls, proxies=proxies):
        '''Проверяет, появились ли на ресурсе новые публикации,
        и если появились, заносит их в план на скачивание.'''
        all_new_urls = nurls[self.source_name]
        page_count = 1
        source = self.source

        attempts_counts = 0
        while True:
            try:
                page_url = source['first_page_url'].replace('1', str(page_count))
                page = requests.get(page_url,
                                    headers=headers,
                                    proxies=proxies,
                                    stream=False)
                soup = BeautifulSoup(page.text, 'html.parser')
                urls = {url['href'] for url in soup.find_all(*source['find_all_args'])}
                if not urls:
                    # the thread processes the case when captcha and no urls
                    print("Changing proxy because of no urls or CAPTCHA...")
                    attempts_counts += 1
                    if attempts_counts >= 2:
                        print(f"Data parsing of {self.source_name} is stopped at {page_url}.")
                        break
                    proxies = None
                    continue
                old_urls = set(self.df.index.to_list())
                new_urls = urls - old_urls
                all_new_urls = all_new_urls | new_urls
                nurls[self.source_name] = all_new_urls
                if len(new_urls) > 0:
                    print(f"{len(new_urls)} urls are collected from {page_url}")
                if (len(new_urls) == 0) or (len(urls) == 0):    
                    print(f"{len(all_new_urls)} new urls are saved for {source['base_url']}")
                    break
                else:
                    page_count += 1
            except ConnectionError:
                print("Problems with connection...")
                continue
        with open('not_processed_urls.pickle', 'wb') as f:
            pickle.dump(nurls, f)
        return nurls


    def save_new_data(self, url, data, full_text):
        '''Сохраняет новые данные в датафрейм, а текст -- в отдельный файл'''
        data['filename'] = str(uuid.uuid5(uuid.NAMESPACE_DNS, url))
        new_row = pd.DataFrame(data, index=[url])
        if url not in self.df:
            self.df = self.df.append(new_row)
        else:
            self.df.iloc[url] = new_row
        
        filepath = f"{DATASETS_PATH}/{self.source_name}/{data['filename']}"
        if not os.path.exists(filepath):
            with open(filepath, 'w') as f:
                full_text = re.sub('\n+', '\n\n', full_text).strip()
                f.write(full_text)
                #print(f"File {self.source_name}/{data['filename']} is saved.")
        else:
            pass
            #print(f"File {self.source_name}/{data['filename']} is already exist.")
            
    def save_df(self):
        self.df.to_csv(f"{DATASETS_PATH}/{self.source_name}.csv")
            
    def concat_csv(self):
        '''
        Соединяет старый csv и тот, что получен из новых данных.
        '''
        pass
    

class HabrParser(Parser):
    '''Парсер для сайта habr.com'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        page = requests.get(url, headers=headers, stream=False)
        soup = BeautifulSoup(page.text, 'html.parser')
        data['title'] = soup.select_one('h1').text.strip()
        try:
            date_time = soup.select_one('.post__time').get('data-time_published')
            data['post_time'] = datetime.datetime.strptime(date_time, '%Y-%m-%dT%H:%MZ')
        except AttributeError:
            date_time = None
        try:
            data['views_num'] = k_to_num(soup.select_one('.post-stats__views-count').text.strip())
        except AttributeError:
            data['views_num'] = 0
        try:
            data['likes_num'] = k_to_num(soup.select_one('.js-post-vote').text.strip().replace('–', '-'))
        except AttributeError:
            data['likes_num'] = 0
        try:
            data['favs_num'] = k_to_num(soup.select_one('.bookmark__counter').text)
        except AttributeError:
            data['favs_num'] = 0
        try:
            data['comments_num'] = k_to_num(soup.select_one('.post-stats__comments-count').text)
        except AttributeError:
            data['comments_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        try:
            full_text = soup.select_one('.post__body_full').text
        except AttributeError:
            full_text = ""
        self.save_new_data(url, data, full_text)


class TprogerParser(Parser):
    '''Парсер для сайта tproger.ru'''
    def update(self, url):
        '''Скачивает или обновляет данные о статье.'''
        data = dict()
        try:
            driver = webdriver.Chrome('/usr/bin/chromedriver')
            driver.get(url)
            page = driver.page_source
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '.post-views-count'))
            WebDriverWait(driver, timeout=10).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        finally:
            driver.quit()

        soup = BeautifulSoup(page, 'html.parser')
        data['title'] = soup.h1.text.strip()
        data['post_time'] = datetime.datetime.fromisoformat(soup.select_one('time')['content'])
        paragraphs = soup.find_all('div', {'class':'entry-content'})[0].find_all('p')
        try:
            data['short_text'] = paragraphs[0].text
        except IndexError:
            data['short_text'] = ''
        try:
            data['views_num'] = soup.select_one('.post-views-count').text
        except AttributeError:
            data['views_num'] = 0
        data['parse_time'] = datetime.datetime.now()
        full_text = "\n\n".join(p.text for p in paragraphs[:-1])
        self.save_new_data(url, data, full_text)    
        
        
class ParserComposer:
    def __init__(self):
        self.parsers = [HabrParser('habr'),
                        TprogerParser('tproger')] 
        with open('not_processed_urls.pickle', 'rb') as f:
            self.nurls = pickle.load(f)
        
    def load_not_processed_articles(self):
        '''Загружает и обрабатывает статьи что не были обработаны
        по каким-то причинам ранее (обрыв сигнала, вызов исключения и т.д.)'''
        
        pairs = [(key, item) for key in self.nurls for item in self.nurls[key]]
        random.shuffle(pairs)
        
        try:
            for pair in tqdm(pairs):
                try:
                    parser = [parser for parser in self.parsers if parser.source_name == pair[0]][0]
                    parser.update(pair[1])
                except ConnectionError:
                    continue
        finally:
            for parser in self.parsers:
                parser.save_df()
                self.nurls[parser.source_name] -= set(parser.df.index)
            with open('not_processed_urls.pickle', 'wb') as f:
                pickle.dump(self.nurls, f)
    
    
    def update_not_old_articles(self):
        for parser in self.parsers:
            n = datetime.datetime.now()
            p = parser.df.post_time.apply(lambda x: x.replace(tzinfo=None))
            #! Второе условие должно быть по парсингу, а не дате выхода статьи
            condition = ((n - p) <= datetime.timedelta(days=30)) & ((n - p) >= datetime.timedelta(days=2))
            parser.not_old = set(parser.df[condition].index)
        pairs = [(parser.source_name, url) for parser in self.parsers for url in parser.not_old]
        random.shuffle(pairs)
        try:
            for pair in tqdm(pairs):
                try:
                    parser = [parser for parser in self.parsers if parser.source_name == pair[0]][0]
                    parser.update(pair[1])
                except ConnectionError:
                    continue
        finally:
            for parser in self.parsers:
                parser.save_df()
        
    
    def scheduler(self):
        '''
        3) загружаем данные новых статей
        -> 4) обновляем число просмотров статей, вышедших в последний месяц (post_time)
        5) обновляем число просмотров статей, проверявшихся > месяца назад (parse_time), не относящихся к п. 2-3
        '''
        pass
    
    def pipeline(self):
        '''Сортирует задачи таким образом, чтобы
        равномерно распределить запросы к внешним ресурсам.
        Проверяем, вышли ли новые статьи. Планируем загрузки. Обновляем датасеты.'''
        print("Load articles not processed before...")
        self.load_not_processed_articles()
        for parser in self.parsers:
            print("Check new articles...")
            self.nurls = parser.check_new_articles(self.nurls)
            print("Update data for not old articles...")
            self.update_not_old_articles()
            
#         with open('not_processed_urls.pickle', 'wb') as f:
#             pickle.dump(self.nurls, f)


In [3]:
composer = ParserComposer()
composer.pipeline()

Load articles not processed before...



Check new articles...
20 urls are collected from https://habr.com/ru/page1/
12 urls are collected from https://habr.com/ru/page2/
32 new urls are saved for https://habr.com/
Update data for not old articles...



Check new articles...
Changing proxy because of no urls or CAPTCHA...
1 urls are collected from https://tproger.ru/page/1/
1 new urls are saved for https://tproger.ru/
Update data for not old articles...
